In [1]:
# read anime-face csv
from os.path import join, isfile
import random
import pandas as pd
import imageio
import os
import numpy as np
from config import ANIME_PATH, ANNOTAION_PATH


def naive_iterative_generator(shuffle=True):

    with open(ANNOTAION_PATH, 'r') as f:
        csv = pd.read_csv(f)
    #     
    img_paths = []
    tags = []
    for row in csv.iterrows():
        path, tag = row[1]
        img_paths.append(path)
        tags.append(tag)
        
#     generator, miss file count 28213, total count 143297.
    N = len(img_paths)
    while True:
        idxs = range(N)
        if shuffle:
            random.shuffle(idxs)
        for idx in idxs:
            path, tag = img_paths[idx], tags[idx]
            path = join(ANIME_PATH, path)
            if isfile(path):
                img = imageio.imread(path)
                yield img, tag
    
def naive_bootsrap_generator():

    with open(ANNOTAION_PATH, 'r') as f:
        csv = pd.read_csv(f)
    #     
    img_paths = []
    tags = []
    for row in csv.iterrows():
        path, tag = row[1]
        img_paths.append(path)
        tags.append(tag)
        
#     generator, miss file count 28213, total count 143297.
    N = len(img_paths)
    while True:
        idx = random.randint(0,N)
        path, tag = img_paths[idx], tags[idx]
        path = join(ANIME_PATH, path)
        if isfile(path):
            img = imageio.imread(path)
            yield img, tag


def img_generator(g):
    batch_size=1
    while True:
        imgs, tags = [],[]
        for i in range(batch_size):
            img, tag = g.next()
            h,w,dep = img.shape
            imgs.append(img.reshape([1, h, w, dep]))
            tags.append(tag)
        batch_size = ( yield np.vstack(imgs), tags )
        
#         
        if batch_size is None:
            batch_size = 1

    
def bootstrap_generator():
    g = naive_bootsrap_generator()
    return img_generator(g)

def iterative_generator(shuffle=True):
    g = naive_iterative_generator(shuffle)
    return img_generator(g)




In [2]:

g = iterative_generator()
img, tag = g.next()


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
img, tag = g.send(6)
print img.shape, tag


(6, 96, 96, 3) ['necklace', 'blush', 'blazer', 'smile', 'uniform', 'comic']
